In [20]:
from agents.medico_base import get_medico, clinical_summary
import json

In [21]:
import os
os.environ["DATAPIZZA_AGENT_LOG_LEVEL"] = "ERROR"

In [22]:
medico = get_medico()
response = medico.run(clinical_summary)

KeyboardInterrupt: 

In [ ]:
piano_terapeutico = json.loads(response.text)

In [ ]:
from agents.nutrizionista import get_nutrizionista

nutrizionista = get_nutrizionista()

response = nutrizionista.run(json.dumps(piano_terapeutico))

piano_settimanale = json.loads(response.text)

Psicologo

In [ ]:
from agents.psicologo import get_psyai, daily_payload, daily_payload_good

psicologo = get_psyai()


input_psicologo = {
    "clinical_summary": clinical_summary,
    "treatment_plan": piano_terapeutico,
    "weekly_meal_plan": piano_settimanale,
    **daily_payload_good
}

In [ ]:
psicologo_response = psicologo.run(json.dumps(input_psicologo))

In [ ]:
psicologo_feedback = json.loads(psicologo_response.text)

status = psicologo_feedback.get("pipeline_status")
doctor_report = psicologo_feedback.get("doctor_report")

In [ ]:
print(f"STATUS: {status}")
if status == 'CRITICAL':
    print(f"Doctor report: {doctor_report}")
else:
    print("All parameters are within normal ranges.")

STATUS: STABLE
All parameters are within normal ranges.


# 🏥 HealthcareMAS Complete Workflow Function

Funzione unificata che incorpora tutto il pipeline medico-nutrizionale-psicologico con possibilità di aggiornamento tramite doctor_report.

In [ ]:
def healthcare_mas_pipeline(
    clinical_summary: str,
    daily_data: dict,
    doctor_report: str = None,
    silent_mode: bool = True
) -> dict:
    """
    🏥 HealthcareMAS Complete Pipeline Function
    
    Esegue l'intero workflow: MedAI → NutriAI → PsyAI
    con possibilità di aggiornamento tramite doctor_report
    
    Args:
        clinical_summary (str): Report clinico del paziente
        daily_data (dict): Dati giornalieri (biometrics + feedback)
        doctor_report (str, optional): Report del medico per aggiornamenti
        silent_mode (bool): Se True, sopprime output degli agenti
        
    Returns:
        dict: Risultato completo con tutti i piani e assessment
    """
    from agents.medico_base import get_medico
    from agents.nutrizionista import get_nutrizionista
    from agents.psicologo import get_psyai
    import json
    import os
    
    # Configura logging silenzioso se richiesto
    if silent_mode:
        os.environ["DATAPIZZA_AGENT_LOG_LEVEL"] = "ERROR"
    
    # Se presente doctor_report, aggiungilo al clinical_summary
    updated_clinical_summary = clinical_summary
    if doctor_report:
        updated_clinical_summary += f"\n\nDOCTOR REPORT (UPDATE):\n{doctor_report}"
    
    try:
        # 🩺 FASE 1: MedAI - Analisi medica e piano terapeutico
        print("🩺 Esecuzione MedAI - Analisi clinica...")
        medico = get_medico()
        medico_response = medico.run(updated_clinical_summary)
        piano_terapeutico = json.loads(medico_response.text)
        
        # 🥗 FASE 2: NutriAI - Piano nutrizionale sincronizzato
        print("🥗 Esecuzione NutriAI - Piano nutrizionale...")
        nutrizionista = get_nutrizionista()
        nutri_response = nutrizionista.run(json.dumps(piano_terapeutico))
        piano_settimanale = json.loads(nutri_response.text)
        
        # 🧠 FASE 3: PsyAI - Assessment psicologico e monitoring
        print("🧠 Esecuzione PsyAI - Assessment psicologico...")
        psicologo = get_psyai()
        
        input_psicologo = {
            "clinical_summary": updated_clinical_summary,
            "treatment_plan": piano_terapeutico,
            "weekly_meal_plan": piano_settimanale,
            **daily_data
        }
        
        psyai_response = psicologo.run(json.dumps(input_psicologo))
        psicologo_feedback = json.loads(psyai_response.text)
        
        # Estrai status e doctor_report
        pipeline_status = psicologo_feedback.get("pipeline_status", "UNKNOWN")
        new_doctor_report = psicologo_feedback.get("doctor_report")
        
        # Risultato completo
        result = {
            "timestamp": "2025-11-22",
            "pipeline_status": pipeline_status,
            "medical_plan": piano_terapeutico,
            "nutrition_plan": piano_settimanale,
            "psychological_assessment": psicologo_feedback,
            "doctor_report_generated": new_doctor_report,
            "requires_medical_review": pipeline_status == "CRITICAL"
        }
        
        # Output finale
        print(f"✅ Pipeline completato - STATUS: {pipeline_status}")
        if pipeline_status == "CRITICAL":
            print(f"⚠️  ATTENZIONE: {new_doctor_report}")
        else:
            print("✅ Tutti i parametri sono nella norma")
            
        return result
        
    except Exception as e:
        print(f"❌ Errore nel pipeline: {str(e)}")
        return {
            "error": str(e),
            "pipeline_status": "ERROR",
            "requires_medical_review": True
        }

## 📋 Esempio di utilizzo con i parametri del notebook

Utilizziamo gli stessi parametri che hai già testato nel notebook

In [ ]:
# Utilizzo con i dati del notebook esistente
from agents.medico_base import clinical_summary
from agents.psicologo import daily_payload

# Esecuzione del pipeline completo
result = healthcare_mas_pipeline(
    clinical_summary=clinical_summary,
    daily_data=daily_payload,
    doctor_report=None,  # Nessun aggiornamento medico
    silent_mode=True
)

🩺 Esecuzione MedAI - Analisi clinica...
🥗 Esecuzione NutriAI - Piano nutrizionale...
🥗 Esecuzione NutriAI - Piano nutrizionale...
🧠 Esecuzione PsyAI - Assessment psicologico...
🧠 Esecuzione PsyAI - Assessment psicologico...
✅ Pipeline completato - STATUS: CRITICAL
⚠️  ATTENZIONE: The patient's physiological metrics indicate significant stress and poor adherence to medication protocols. Sleep deprivation may impact overall health and medication efficacy, necessitating a re-evaluation of the treatment plan.
✅ Pipeline completato - STATUS: CRITICAL
⚠️  ATTENZIONE: The patient's physiological metrics indicate significant stress and poor adherence to medication protocols. Sleep deprivation may impact overall health and medication efficacy, necessitating a re-evaluation of the treatment plan.


In [ ]:
# Visualizza il risultato strutturato
print("📊 RISULTATI DEL PIPELINE:")
print("=" * 50)
print(f"Status: {result['pipeline_status']}")
print(f"Revisione medica richiesta: {result['requires_medical_review']}")
print(f"Timestamp: {result['timestamp']}")

if result.get('doctor_report_generated') and result['doctor_report_generated'] != 'None':
    print(f"\n⚠️ Doctor Report: {result['doctor_report_generated']}")

# Accesso ai singoli piani
medical_plan = result['medical_plan']
nutrition_plan = result['nutrition_plan']
psych_assessment = result['psychological_assessment']

print(f"\n🩺 Piano medico generato con {len(medical_plan['treatment_plan'])} farmaci")
print(f"🥗 Piano nutrizionale per 7 giorni")
print(f"🧠 Assessment psicologico: {psych_assessment.get('adherence_score', 'N/A')}")

📊 RISULTATI DEL PIPELINE:
Status: CRITICAL
Revisione medica richiesta: True
Timestamp: 2025-11-22

⚠️ Doctor Report: The patient's physiological metrics indicate significant stress and poor adherence to medication protocols. Sleep deprivation may impact overall health and medication efficacy, necessitating a re-evaluation of the treatment plan.

🩺 Piano medico generato con 4 farmaci
🥗 Piano nutrizionale per 7 giorni
🧠 Assessment psicologico: MEDIUM


## 🔄 Esempio con Doctor Report (aggiornamento)

Simuliamo un caso in cui il medico fornisce un aggiornamento al piano terapeutico

In [ ]:
# Simulazione di un doctor report per aggiornamento terapia
doctor_update = """
MEDICAL UPDATE - Dr. Smith Review (2025-11-22):

After reviewing the initial treatment plan and patient's response:
1. Metformin dosage should be increased to 1000mg twice daily due to persistent HbA1c levels
2. Add Atorvastatin 20mg daily for cholesterol management 
3. Patient reports good tolerance to current medications
4. Continue current GERD treatment but monitor for effectiveness
5. Schedule follow-up in 2 weeks to assess glucose control

PRIORITY: Monitor for any signs of lactic acidosis with increased Metformin dose.
"""

if result["requires_medical_review"]:
    print("\n⚠️ Il pipeline richiede una revisione medica basata sui dati forniti.")
    doctor_update = result["doctor_report_generated"]

# Nuova esecuzione con doctor_report
from agents.psicologo import daily_payload  # Usiamo scenario più critico

result_updated = healthcare_mas_pipeline(
    clinical_summary=clinical_summary,
    daily_data=daily_payload,  # Dati meno ottimali
    doctor_report=doctor_update,  # Aggiornamento del medico
    silent_mode=True
)

🩺 Esecuzione MedAI - Analisi clinica...
🥗 Esecuzione NutriAI - Piano nutrizionale...
🥗 Esecuzione NutriAI - Piano nutrizionale...
🧠 Esecuzione PsyAI - Assessment psicologico...
🧠 Esecuzione PsyAI - Assessment psicologico...
✅ Pipeline completato - STATUS: CRITICAL
⚠️  ATTENZIONE: Critical issues include missed Metformin doses, which could lead to poor glucose management and uncontrolled symptoms. The low HRV and sleep may suggest increasing stress levels affecting your overall health. Immediate re-evaluation of your routine and treatment is necessary.
✅ Pipeline completato - STATUS: CRITICAL
⚠️  ATTENZIONE: Critical issues include missed Metformin doses, which could lead to poor glucose management and uncontrolled symptoms. The low HRV and sleep may suggest increasing stress levels affecting your overall health. Immediate re-evaluation of your routine and treatment is necessary.


In [ ]:
# Confronto dei risultati
print("🔍 CONFRONTO RISULTATI:")
print("=" * 60)

print(f"📊 ESECUZIONE ORIGINALE:")
print(f"   Status: {result['pipeline_status']}")
print(f"   Farmaci prescritti: {len(result['medical_plan']['treatment_plan'])}")

print(f"\n📊 ESECUZIONE CON DOCTOR REPORT:")
print(f"   Status: {result_updated['pipeline_status']}")
print(f"   Farmaci prescritti: {len(result_updated['medical_plan']['treatment_plan'])}")

# Mostra differenze nei farmaci
print(f"\n💊 CONFRONTO TERAPIE:")
for i, med in enumerate(result_updated['medical_plan']['treatment_plan']):
    print(f"   {i+1}. {med['medication']} - {med['dosage']} ({med['frequency']})")

if result_updated.get('doctor_report_generated') != 'None':
    print(f"\n⚠️ NUOVO DOCTOR REPORT GENERATO:")
    print(f"   {result_updated.get('doctor_report_generated')}")

🔍 CONFRONTO RISULTATI:
📊 ESECUZIONE ORIGINALE:
   Status: STABLE
   Farmaci prescritti: 4

📊 ESECUZIONE CON DOCTOR REPORT:
   Status: CRITICAL
   Farmaci prescritti: 3

💊 CONFRONTO TERAPIE:
   1. Metformin - 1000 mg (Twice daily)
   2. Atorvastatin - 20 mg (Once daily)
   3. Esomeprazole - 40 mg (Once daily)

⚠️ NUOVO DOCTOR REPORT GENERATO:
   Critical issues include missed Metformin doses, which could lead to poor glucose management and uncontrolled symptoms. The low HRV and sleep may suggest increasing stress levels affecting your overall health. Immediate re-evaluation of your routine and treatment is necessary.
